Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.39.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-192600
aml-quickstarts-192600
southcentralus
81cefad3-d2c9-4f77-a466-99a7f541c7bb


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'assign2-experiment1'
project_folder = './pipeline-bank'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
assign2-experiment1,quick-starts-ws-192600,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "assign2-cluster1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "bankmarketingHK2"
description_text = "dataset assignment 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Azure/MachineLearningNotebooks/master/how-to-use-azureml/automated-machine-learning/forecasting-bike-share/bike-no.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [ ]:
dataset.take(5).to_pandas_dataframe()

## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `time_column_name` has to be `cnt` for example.

In [10]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",  
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [11]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [12]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [13]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [14]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [38e157ee][434d81d0-e96f-4c80-8ba2-3af54af5ff33], (This step will run and generate new outputs)
Submitted PipelineRun 71308065-71d3-400b-abfb-97565f06c12d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/71308065-71d3-400b-abfb-97565f06c12d?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-192600/workspaces/quick-starts-ws-192600&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [15]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [16]:
pipeline_run.wait_for_completion()

PipelineRunId: 71308065-71d3-400b-abfb-97565f06c12d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/71308065-71d3-400b-abfb-97565f06c12d?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-192600/workspaces/quick-starts-ws-192600&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: Running


StepRunId: 22df57f1-07c7-46e9-bc1b-742e7e84fe0d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/22df57f1-07c7-46e9-bc1b-742e7e84fe0d?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-192600/workspaces/quick-starts-ws-192600&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 20 minute(s)
{'runId': '22df57f1-07c7-46e9-bc1b-742e7e84fe0d', 'target': 'assign2-cluster1', 'stat

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [17]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/22df57f1-07c7-46e9-bc1b-742e7e84fe0d/metrics_data, 1 files out of an estimated total of 1


In [18]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_4,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_0,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_1,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_2,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_17,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_15,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_3,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_14,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_18,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_21,...,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_23,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_27,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_29,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_31,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_26,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_28,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_30,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_24,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_32,22df57f1-07c7-46e9-bc1b-742e7e84fe0d_35
AUC_micro,[0.9732255383035407],[0.9795361989126856],[0.9783641467160662],[0.9621052728532908],[0.8917366405622167],[0.9775185191155036],[0.9658041682689319],[0.9778851480953575],[0.978789217119791],[0.978205355518662],...,[0.876932585123457],[0.9780501564655144],[0.9779279314545191],[0.9793589864626819],[0.9762675318514971],[0.9780202219300406],[0.9783447122945742],[0.979105694239444],[0.9769312495826433],[0.9786410181426312]
recall_score_weighted,[0.9071320182094081],[0.9101669195751139],[0.9116843702579667],[0.898937784522003],[0.8033383915022762],[0.9068285280728376],[0.8880121396054628],[0.9141122913505311],[0.9132018209408195],[0.9104704097116844],...,[0.775720789074355],[0.9101669195751139],[0.9132018209408195],[0.9125948406676783],[0.9089529590288316],[0.9110773899848255],[0.9059180576631259],[0.9162367223065251],[0.9074355083459787],[0.9104704097116844]
norm_macro_recall,[0.2915131509483242],[0.48912840119515355],[0.4955737458946703],[0.16624062002752638],[0.674333653794501],[0.22012347943028288],[0.0],[0.4106821006692636],[0.49254603619173576],[0.45631447428623284],...,[0.7000715017403079],[0.4464996563841237],[0.38597417416416135],[0.5131768815979343],[0.3930308031720098],[0.4617345284867749],[0.2759365153460147],[0.4414936083742247],[0.4315824668841357],[0.4610510014874585]
log_loss,[0.22090578349014284],[0.17851374134751752],[0.18692041766923378],[0.2629525672764293],[0.4228172733537645],[0.21258977849913452],[0.2704273105907245],[0.20615413680244937],[0.19693610768213313],[0.22587372849244478],...,[0.44824751017682674],[0.1876885204442146],[0.33199296763411795],[0.1804632387509369],[0.19873978109892296],[0.18628352407953988],[0.19986862844075845],[0.200473506044025],[0.19393005534183488],[0.18445374122721345]
f1_score_micro,[0.9071320182094081],[0.9101669195751139],[0.9116843702579667],[0.8989377845220029],[0.8033383915022762],[0.9068285280728376],[0.8880121396054628],[0.9141122913505311],[0.9132018209408195],[0.9104704097116844],...,[0.775720789074355],[0.9101669195751139],[0.9132018209408195],[0.9125948406676783],[0.9089529590288316],[0.9110773899848255],[0.9059180576631259],[0.9162367223065252],[0.9074355083459787],[0.9104704097116844]
f1_score_weighted,[0.8909573471197098],[0.9072831557855964],[0.9087877883554606],[0.871192710611544],[0.8353561666850935],[0.883038347285473],[0.8353395018439429],[0.9056260494307121],[0.9098016443897835],[0.9057003286914337],...,[0.815057223742878],[0.9048928710960408],[0.9032152403319238],[0.9103870858100789],[0.900539981658476],[0.9064755047110378],[0.8886031510001888],[0.9092400519650629],[0.9019283088360116],[0.9059729706274329]
recall_score_macro,[0.6457565754741621],[0.7445642005975768],[0.7477868729473351],[0.5831203100137632],[0.8371668268972505],[0.6100617397151414],[0.5],[0.7053410503346318],[0.7462730180958679],[0.7281572371431164],...,[0.8500357508701539],[0.7232498281920618],[0.6929870870820807],[0.7565884407989671],[0.6965154015860049],[0.7308672642433874],[0.6379682576730074],[0.7207468041871123],[0.7157912334420679],[0.7305255007437292]
matthews_correlation,[0.42149687086703563],[0.5216286298277554],[0.52924365161229],[0.3149058510980744],[0.4726176661690189],[0.3934019612341506

### Retrieve the Best Model

In [19]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/22df57f1-07c7-46e9-bc1b-742e7e84fe0d/model_data, 1 files out of an estimated total of 1


In [20]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/m...
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('31', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.7, eta=0.4, gamma=5, max_depth=6, max_leaves=0, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_al

In [21]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('31', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.7, eta=0.4, gamma=5, max_depth=6, max_leaves=0, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=1.7708333333333335, reg_lambda=1.5625, subsample=0.5, tree_method='auto'))], verbose=False)), ('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [22]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing HK Train", description="Training bankmarketing HK pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing HK Train,285ea976-206b-43c7-86b3-16b664b186ed,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [23]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [30]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-HK-rest-endpoint"}
                        )

In [31]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  5f4914fd-cc07-4143-8715-bd1699be6bcd


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [32]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-HK-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …